In [ ]:
import torch
from torch.utils.data import DataLoader

from data import load_data
from visualize import plot_outcome_distribution

from transformers import ViTImageProcessor, ViTForImageClassification

In [ ]:
train = load_data(environment='train', 
                    path_dir="./data/", 
                    generate=False)
train

In [ ]:
def encoder(x, model, processor):
    inputs = processor(images=x, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states[-1][:,0]  

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
train = train.map(lambda x: {"emb1": encoder(x['image'], model, processor)}, batch_size=100, batched=True, num_proc=4)

In [ ]:
image = train[0]['image']

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits
# print("Top 5 predicted labels with associated probabilities:")
# top_5 = torch.topk(logits, 5)
# probs = logits.softmax(-1)[0][top_5.indices][0]
# for i, (idx, prob) in enumerate(zip(top_5.indices[0], probs), 1):
#     print(f"    {i}. {model.config.id2label[idx.item()]}: {prob.item():.2%}")
# model predicts one of the 1000 ImageNet classes

inputs = processor(images=input, return_tensors="pt")
outputs = model(**inputs, output_hidden_states=True)
outputs.hidden_states[-1][:,0].shape

In [ ]:
iterable_dataset = IterableDataset.from_generator(generator, gen_kwargs={"reduce_fps_factor": 10, "downscale_factor": 0.5})


In [ ]:
dataloader = DataLoader(dataset, batch_size=3, num_workers=0)
for batch in dataloader:
    print(batch)
    break

In [ ]:
batch['image'].shape

In [ ]:
# From a generator function
def my_generator(n, sources):
    for source in sources:
        for example_id_for_current_source in range(n):
            yield {"example_id": f"{source}_{example_id_for_current_source}"}

gen_kwargs = {"n": 10, "sources": [f"path/to/data_{i}" for i in range(1024)]}
my_iterable_dataset = IterableDataset.from_generator(my_generator, gen_kwargs=gen_kwargs)
my_iterable_dataset.n_shards  # 1024

In [ ]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.pretrained_encoder = torchvision.models.resnet50(pretrained=True).eval().requires_grad_(False)
        self.linear = nn.Linear(1000, 10)

def forward(x):
    with torch.no_grad():
        emb1 = self.pretrained_encoder(x['image'])
    logits = self.linear(emb1)
    